# Итоговое задание по Проекту 5. Выбираем авто выгодно
***
### Импорт библиотек

In [3]:
import numpy as np 
import pandas as pd 
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
import os

In [4]:
RANDOM_SEED = 42
!pip freeze > requirements.txt
CURRENT_DATE = pd.to_datetime('25/11/2020')

Зададим необходимые функции сразу

### Импорт данных

In [5]:
from importlib import reload
print(os.listdir('./'))
DIR_TRAIN  = '../' # подключил к ноутбуку свой внешний датасет
DIR_TEST   = '../'

['.ipynb_checkpoints', 'all_car_brands_and_their_models_from_auto_ru.csv', 'baseline-sf-dst-car-price-prediction-v14.ipynb', 'Car Price prediction-Copy1.ipynb', 'Car Price prediction.ipynb', 'requirements.txt', 'sample_submission.csv', 'test.csv', 'train.csv', 'Untitled.ipynb']


### Первичная обработка датасетов

In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submit = pd.read_csv('sample_submission.csv')
pd.set_option('display.max_columns', None)
print('Размерность тренировочного датасета: ', train.shape)
display(train.head(2))
print('Размерность тестового датасета: ', test.shape)
display(test.head(2))
print('Размерность датасета c примером сабмишена: ', submit.shape)
display(submit.head(2))

Размерность тренировочного датасета:  (89378, 26)


,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,start_date,hidden,model
0,Седан,AUDI,040001,бензин,1990.0,2.8 MT (174 л.с.) 4WD,4.0,1991,SEDAN MECHANICAL 2.8,MECHANICAL,2.8,174.0,"Машина в приличном состоянии ,не гнилая не р...",350000,{'id': '0'},полный,LEFT,NaN,3.0,ORIGINAL,True,NaN,200000.0,2019-10-03T08:09:11Z,NaN,100
1,Седан,AUDI,EE1D19,бензин,1982.0,1.8 MT (90 л.с.),4.0,1986,SEDAN MECHANICAL 1.8,MECHANICAL,1.8,90.0,Машина в оригинале не гнилая все вопросы по те...,173424,{'id': '0'},передний,LEFT,NaN,3.0,ORIGINAL,True,NaN,60000.0,2020-09-06T06:49:40Z,NaN,100


Размерность тестового датасета:  (3837, 23)


,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,id
0,седан,BMW,чёрный,дизель,2016.0,520d 2.0d AT (190 л.с.),4.0,2017.0,SEDAN AUTOMATIC 2.0,автоматическая,2.0 LTR,190 N12,В РОЛЬФ Ясенево представлено более 500 автомоб...,158836.0,"['[{""name"":""Безопасность"",""values"":[""Антипробу...",задний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,0
1,седан,BMW,белый,дизель,2018.0,318d 2.0d AT (150 л.с.),4.0,2019.0,SEDAN AUTOMATIC 2.0,автоматическая,2.0 LTR,150 N12,Автомобиль из демонстрационного парка по спец ...,10.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",задний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,1


Размерность датасета c примером сабмишена:  (3837, 2)


,id,price
0,0,5299000.0
1,1,1580000.0


In [7]:
def check_df_before_merg(d_df1,d_df2):
    
    list_of_names1 = list(d_df1.columns)
    temp_dict = {}
    temp_dict['Уникальных_1'] = d_df1.nunique().values
    temp_dict['Первая_строка_1'] =d_df1.loc[0].values
    temp_dict['Тип_1'] = d_df1.dtypes
    temp_dict['Признак_1'] = list_of_names1
    temp_df1 = pd.DataFrame.from_dict(temp_dict)
    
    
    list_of_names2 = list(d_df2.columns)
    temp_dict2 = {}
    temp_dict2['Признак_2'] = list_of_names2
    temp_dict2['Тип_2'] = d_df2.dtypes
    temp_dict2['Первая_строка_2'] =d_df2.loc[0].values
    temp_dict2['Уникальных_2'] = d_df2.nunique().values
    temp_df2 = pd.DataFrame.from_dict(temp_dict2)
    
    temp_df = pd.concat([temp_df1,temp_df2], axis=1, sort=False)
    temp_df.reset_index(inplace = True)
    del temp_df['index']
    display(temp_df)

    temp_dict3 = {}
    temp_df3= pd.DataFrame(temp_df)
    temp_list  = []
    temp_list2  = []
    temp_list3  = []
    temp_list4  = []
    temp_list5  = []

    for i in range(len(temp_df)):
        if str(temp_df3['Тип_2'][i]) != str(temp_df3['Тип_1'][i]):
            temp_list.append(temp_df3['Признак_1'][i])
            temp_list2.append(temp_df3['Признак_2'][i])
            temp_list3.append(str(temp_df3['Тип_1'][i]) + '!=' + str(temp_df3['Тип_2'][i]))
            temp_list4.append(i)
        if temp_df3['Уникальных_2'][i]>0 and temp_df3['Уникальных_1'][i]/temp_df3['Уникальных_2'][i] > 2:
            temp_list5.append(i)
            
    temp_dict3['index']= temp_list4
    temp_dict3['Признак_1']= temp_list
    temp_dict3['не совпадают типы'] = temp_list3
    temp_dict3['Признак_2']= temp_list2

    temp_df4 = pd.DataFrame.from_dict(temp_dict3)
    temp_df4.set_index('index',inplace=True)

    print(f'Резюме:\n1. Не совпали типы в:= {len(temp_df4)} столбцах\n')
    print(f'2. Уникальные значения заоблачно различаются в:= {len(temp_list5)} столбцах {temp_list5}')
    display(temp_df4)

    return

In [8]:
d_df1 = train
d_df2 = test
list_of_names1 = list(d_df1.columns)
temp_dict = {}
temp_dict['Уникальных_1'] = d_df1.nunique().values
temp_dict['Первая_строка_1'] =d_df1.loc[0].values
temp_dict['Тип_1'] = d_df1.dtypes
temp_dict['Признак_1'] = list_of_names1
temp_df1 = pd.DataFrame.from_dict(temp_dict)
    
    
list_of_names2 = list(d_df2.columns)
temp_dict2 = {}
temp_dict2['Признак_2'] = list_of_names2
temp_dict2['Тип_2'] = d_df2.dtypes
temp_dict2['Первая_строка_2'] =d_df2.loc[0].values
temp_dict2['Уникальных_2'] = d_df2.nunique().values
temp_df2 = pd.DataFrame.from_dict(temp_dict2)
    
temp_df = pd.concat([temp_df1,temp_df2], axis=1, sort=False)
temp_df.reset_index(inplace = True)
del temp_df['index']
display(temp_df)

,Уникальных_1,Первая_строка_1,Тип_1,Признак_1,Признак_2,Тип_2,Первая_строка_2,Уникальных_2
0,168.0,Седан,object,bodyType,bodyType,object,седан,11.0
1,36.0,AUDI,object,brand,brand,object,BMW,1.0
2,16.0,040001,object,color,color,object,чёрный,15.0
3,6.0,бензин,object,fuelType,fuelType,object,дизель,4.0
4,78.0,1990,float64,modelDate,modelDate,float64,2016,31.0
5,4484.0,2.8 MT (174 л.с.) 4WD,object,name,name,object,520d 2.0d AT (190 л.с.),384.0
6,5.0,4,float64,numberOfDoors,numberOfDoors,float64,4,4.0
7,80.0,1991,int64,productionDate,productionDate,float64,2017,37.0
8,1860.0,SEDAN MECHANICAL 2.8,object,vehicleConfiguration,vehicleConfiguration,object,SEDAN AUTOMATIC 2.0,99.0
9,4.0,MECHANICAL,object,vehicleTransmission,vehicleTransmission,object,автоматическая,3.0


In [9]:
def nunique_not_found(d_df1, d_df2, d_col):

    temp_set = set()
    temp_set2 = set()
    temp_set = set(d_df1[d_col].unique())
    temp_set2 = set(d_df2[d_col].unique())

    print(f'в столбце:= {d_col} в тренировночном датасете := {len(temp_set2-temp_set)} уникальных значений из тестового')

    return list(temp_set2-temp_set)

In [10]:
# сравним датасеты 
check_df_before_merg(train, test)

,Уникальных_1,Первая_строка_1,Тип_1,Признак_1,Признак_2,Тип_2,Первая_строка_2,Уникальных_2
0,168.0,Седан,object,bodyType,bodyType,object,седан,11.0
1,36.0,AUDI,object,brand,brand,object,BMW,1.0
2,16.0,040001,object,color,color,object,чёрный,15.0
3,6.0,бензин,object,fuelType,fuelType,object,дизель,4.0
4,78.0,1990,float64,modelDate,modelDate,float64,2016,31.0
5,4484.0,2.8 MT (174 л.с.) 4WD,object,name,name,object,520d 2.0d AT (190 л.с.),384.0
6,5.0,4,float64,numberOfDoors,numberOfDoors,float64,4,4.0
7,80.0,1991,int64,productionDate,productionDate,float64,2017,37.0
8,1860.0,SEDAN MECHANICAL 2.8,object,vehicleConfiguration,vehicleConfiguration,object,SEDAN AUTOMATIC 2.0,99.0
9,4.0,MECHANICAL,object,vehicleTransmission,vehicleTransmission,object,автоматическая,3.0


Резюме:
1. Не совпали типы в:= 11 столбцах

2. Уникальные значения заоблачно различаются в:= 10 столбцах [0, 1, 4, 5, 7, 8, 10, 11, 12, 13]


,Признак_1,не совпадают типы,Признак_2
index,,,
7,productionDate,int64!=float64,productionDate
11,enginePower,float64!=object,enginePower
13,mileage,int64!=float64,mileage
17,Состояние,float64!=object,Состояние
18,Владельцы,float64!=object,Владельцы
20,Таможня,bool!=object,Таможня
22,price,float64!=nan,NaN
23,start_date,object!=nan,NaN
24,hidden,float64!=nan,NaN


In [11]:
# на первый взгляд может, что перевод во float достаточно бесмыссленный шаг ведь кол-во дверей автомобиля это целое число. Но напомню что мы сейчас на этапе приведения датасета после парсинга к виду тестового. На этапе анализа датасета мы сможем более детально посмотреть что происходит в тестовом датасете (вероятно там есть ошибки которые надо исправить и мы можем потерять эту информацию не знаю сколько ошибок и какие они) 

# начнем с int64!=float64 и приведем все к float 
list_cols_to_repair = ['productionDate', 'mileage']
for col in list_cols_to_repair:
        train[col] = train[col].astype('float64')

# теперь починим int64!=float64 и приведем все к object
list_cols_to_repair = ['enginePower', 'Состояние', 'Владельцы']
for col in list_cols_to_repair:
        train[col] = train[col].astype('object')

# теперь починим bool!=object

train['Таможня'] = train['Таможня'].map({True: 'Растаможен', False:'Не растаможен'})

In [ ]:
# сравним датасеты 
check_df_before_merg(train, test)

Резюме - несоответствие типов осталось в 5 столбцах [price, start_date, hidden, model, id], в том числе price - целевая переменная которую мы будем моделировать, три столбца чтобы попробовать обогатить данные, и столбцец id, который нам будет нужен для формирования файла submition.
Теперь обратим внимание на значения и приведем их к общему виду

In [ ]:
# bodyType - надо привести значения к нижнему регистру (оставим закадром, проверку уникальных значений трейна и теста, чтобы прийти к такому выводу)
train['bodyType'] = train['bodyType'].apply(lambda x: str(x).lower())
nunique_not_found(train, test, 'bodyType')


# color - цвета спарсились в шестнадцатеричном коде цветов. Но их всего 16 поэтому воспользуемся сайтом (https://hysy.org/color) и сделаем словарь цветов. (оставим закадром, проверку уникальных значений трейна и теста, чтобы сделать соответсвия цветов) 
dict_color = {'040001':'чёрный', 'EE1D19':'красный', '0000CC':'синий', 
              'CACECB':'серебристый', '007F00':'зелёный', 'FAFBFB':'белый', 
              '97948F':'серый', '22A0F8':'голубой', '660099':'пурпурный', 
              '200204':'коричневый', 'C49648':'бежевый', 'DEA522':'золотистый', 
              '4A2197':'фиолетовый', 'FFD600':'жёлтый', 'FF8649':'оранжевый', 
              'FFC0CB':'розовый'}
train['color'] = train['color'].map(dict_color)
nunique_not_found(train, test, 'color')

# vehicleTransmission - тип трасмиссии по словарю из 4 значений
train['vehicleTransmission'] = train['vehicleTransmission'].map({'MECHANICAL':'механическая', 'AUTOMATIC':'автоматическая', 'ROBOT':'роботизированная', 'VARIATOR':'вариатор'})


# Руль - ну тут все просто - Правый или Левый в нижнем регистре с большой буквы
train['Руль'] = train['Руль'].map({'RIGHT':'Правый', 'LEFT':'Левый'})
nunique_not_found(train, test, 'Руль')

# ПТС - тут все просто - Оригинал или Дубликат в нижнем регистре с большой буквы
train['ПТС'] = train['ПТС'].map({'ORIGINAL':'Оригинал', 'DUPLICATE':'Дубликат'})
nunique_not_found(train, test, 'ПТС')

# Владельцы - Оригинал или Дубликат в нижнем регистре с большой буквы
train['Владельцы'] = train['Владельцы'].map({3.0:'3 или более', 2.0:'2\xa0владельца', 1.0:'1\xa0владелец'})
nunique_not_found(train, test, 'Владельцы')

# Владение - в таблице сверху в первых строках Nan поэтому нужно отдельно проверить
# оказалось что в тесте очень сложная строковая запись тогда как в трейне в значениях более структурируемые словари (УчТЕМ этот момент и разберем этот столбец на этапе анализа, вероятно будет сразу полезно создать дополнительные временные фичи) 
nunique_not_found(train, test, 'Владение');

Резюме - несоответсвие значений в одинаковых типах осталось только в одном столбце - Владение, но приведение к единому виду целеообразно оставить до детально анализа по переменным, так как там зашиты данные о годе и месяце владения и в процессе обработке может быть сразу целесообразно будет создавать новые временные фичи.

### Слияние датасетов в один

In [12]:
# ВАЖНО! для корректной обработки признаков объединяем трейн и тест в один датасет
train['Train'] = 1 # помечаем где у нас трейн
test['Train'] = 0 # помечаем где у нас тест

df = train.append(test, sort=False).reset_index(drop=True) # объединяем

In [13]:
# проверка  после слияния
df.head(2)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,start_date,hidden,model,Train,id
0,Седан,AUDI,040001,бензин,1990.0,2.8 MT (174 л.с.) 4WD,4.0,1991.0,SEDAN MECHANICAL 2.8,MECHANICAL,2.8,174,"Машина в приличном состоянии ,не гнилая не р...",350000.0,{'id': '0'},полный,LEFT,NaN,3,ORIGINAL,Растаможен,NaN,200000.0,2019-10-03T08:09:11Z,NaN,100,1,NaN
1,Седан,AUDI,EE1D19,бензин,1982.0,1.8 MT (90 л.с.),4.0,1986.0,SEDAN MECHANICAL 1.8,MECHANICAL,1.8,90,Машина в оригинале не гнилая все вопросы по те...,173424.0,{'id': '0'},передний,LEFT,NaN,3,ORIGINAL,Растаможен,NaN,60000.0,2020-09-06T06:49:40Z,NaN,100,1,NaN


### Предварительный анализ данных

In [20]:
def describe_cols(df):
    list_ = list(df.columns)
    temp_dict = {}
    temp_dict['Признак'] = list_
    temp_dict['Тип'] = df.dtypes
    temp_dict['Кол-во значений'] = df.describe(include='all').loc['count']
    temp_dict['Кол-во пропусков(NaN)'] = df.isnull().sum().values 
    temp_dict['Кол-во уникальных'] = df.nunique().values
    temp_df = pd.DataFrame.from_dict(temp_dict, orient='index')
    display(temp_df.T)

    return

In [21]:
describe_cols(df[df['Train']==1])

,Признак,Тип,Кол-во значений,Кол-во пропусков(NaN),Кол-во уникальных
0,bodyType,object,89377,1,168
1,brand,object,89378,0,36
2,color,object,89378,0,16
3,fuelType,object,89378,0,6
4,modelDate,float64,89377,1,78
5,name,object,89377,1,4484
6,numberOfDoors,float64,89377,1,5
7,productionDate,float64,89378,0,80
8,vehicleConfiguration,object,89377,1,1860
9,vehicleTransmission,object,89377,1,4


In [22]:
describe_cols(df[df['Train']==0])

,Признак,Тип,Кол-во значений,Кол-во пропусков(NaN),Кол-во уникальных
0,bodyType,object,3837,0,11
1,brand,object,3837,0,1
2,color,object,3837,0,15
3,fuelType,object,3837,0,4
4,modelDate,float64,3837,0,31
5,name,object,3837,0,384
6,numberOfDoors,float64,3837,0,4
7,productionDate,float64,3837,0,37
8,vehicleConfiguration,object,3837,0,99
9,vehicleTransmission,object,3837,0,3


Резюме по предварительному анализу: (выводы идут по порядку выявления, а не по значимости)
В тренировочной выборке 88968 строк, в тестовой 3837. Всего данных - 93215 строк. Всего 28 признаков, из них:

23 в тесте:  
4 - числовых - mileage, modelDate, productionDate (один из них id - сервисный для генерации submition),  
19 - категориальных (пока так, во время анализа, вероятно, какие-нибудь из них станут числовыми, например можность двигателя)  
4 были добавлены во время парсинга: 1- числовой - price, 1 - временной ряд - start_date, 1 - бинарный - hidden, 1 -   категориальный - model  
1 - бинарный Train - сервисный для разделения train и test  
Детальная информация по трейну и тесту раздельно:  

признак Таможня бесполезен так как содержит одно значение - Растаможен => удалим  
в трейне 2114 (2.4%) дубликатов по строкам => надо удалить  
высокая корреляция между modelDate и productionDate => надо еще внимательно посмотреть на этапе анализа перед тем как удалить
пропуски в трейне:  
description - 3254 (3.6%) => надо заполнить  
Состояние - 89378 (100.0%) - мы не смогли во время парсинга раздобыть этот критерий => надо проанализировать заполненность этого признака в тесте и постараться заполнить по похожим критериям авто требующих ремонта - так как это влияет на цену
Владельцы - 13588 (15.2%) => надо заполнить  
Владение - 60177 (67.3%) => очень много пропусков надо попробовать заполнить  
hidden - 89378 (100.0%) => надо удалить  
id - 89378 (100.0%) - это столбец для генерации файла submition => ничего не делаем  
пропуски в тесте:  
Владение - 2084 (54.3%) => надо заполнить  
price, start_date, hidden, model - 3837 (100.0%) - дополнительные фичи => надо проанализировать и использовать для генерации новых фич или удалить  
между vehicleConfiguration и еще тремя признаками: vehicleTransmission, engineDisplacement, numberOfDoors - высокая корреляция => надо проверить это при анализе и при необходимости удалить  
в тесте:  
brand - только одно значение - "BMW" => надо дополнительно собрать информацию с обзоров цен на BMW и постараться оставить для анализа автомобили со схожими характеристиками, которые влюяют на цену  
Руль - только одно значение - "Левый" => надо взять из трейна только авто с Левым рулем  
Состояние - только одно значение - "Не требует ремонта" => надо обратить внимание на выбросы снизу по цене из трейна, чтобы не брать в учет авто требующие ремонта  
Описания полей датасета  

bodyType - тип кузова автомобиля (https://ru.wikipedia.org/wiki/Типы_автомобильных_кузовов)
brand - марка (бренд) автомобиля  
color - цвет автомобиля (16 цветов: чёрный, красный, синий, серебристый, зелёный, белый, серый, голубой, пурпурный, коричневый, золотистый, фиолетовый, жёлтый, оранжевый, розовый)  
fuelType - тип топлива (5 видов топлива: бензин, дизель, электро, гибрид, газ)  
modelDate - дата выпуска модели (не путать с датой производства)  
name - комбинация нескольких характеристик (модели, объема двигателя и привода)  
numberOfDoors - кол-во дверей  
productionDate - дата производства авто  
vehicleConfiguration - комбинация типа трансмиссии, объема двигателя и мощности двигателя  
vehicleTransmission - тип трансмиссии (коробки передач) (4 вида: автоматическая, механическая, роботизированная, вариатор)  
engineDisplacement - объем двигателя в литрах  
enginePower - мощность двигателя (л.с.)  
description - дополнительные характеристики по комплектации авто  
mileage - пробег автомобиля (км.) - представляет собой общее количество километров, которое это транспортное средство проехало по дорогам с момента своего схода с конвейера производителя.  
Комплектация - дополнительные характеристики по комплектации авто  
Привод - характеристика привода автомобиля, которое передает энергию от двигателя на колеса. Переднеприводные автомобили   получают всю энергию двигателя на передние колеса. При заднем приводе энергия двигателя целиком поступает на задние колеса.   Когда энергия двигателя передается на все четыре колеса автомобиля, такой привод является полным ('полный', 'передний', 'задний')  
Руль - характеристика руля по стороне расположения руля в автомобиле (Левый, Правый)  
Состояние - характеристика (необходимости проведения ремонта - требует ремонта или не требует ремонта)  
Владельцы - кол-во владельцев (3 значения: 1 владелец, 2 владельца, 3 и более)  
ПТС - тип ПТС (паспорт технического средства)(2 значения - оригинал, дубликат)  
Таможня - информация о том расстаможен автомобиль или нет (2 значения - Растаможен, Не ратаможен)  
Владение - срок владения автомобилем в годах и месяцах  
price - цена указанная в объявлении на авто.ру  
start_date - дата размещения объявления на авто.ру  
hidden - характеристика объявления на авто.ру  
model - название модели автомобиля (не путать с маркой (брендом), например: бренд или марка - BMW, модель - M6)  
Train - сервисный для разделения train и test   
id - - сервисный для генерации submition  

In [ ]:
# внесем данные по типам переменных из резюме в списки 
# временной ряд (1)
time_cols = ['start_date']
# бинарная переменная (0) (hidden - не включаем так как решили удалить) итого  (0+1=1)
bin_cols = []
# категориальные переменные (19), ('Таможня' - не включаем  так как решили удалить) (19+1=20)
cat_cols = ['bodyType', 'brand', 'color', 'fuelType', 'name',
       'numberOfDoors', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'description', 'Комплектация', 'Привод', 'Руль', 'Состояние',
       'Владельцы', 'ПТС', 'Владение', 'model']
# числовые переменные (3) 
num_cols = ['mileage', 'modelDate', 'productionDate']
# сервисные переменные (2)
servis_cols = ['Train', 'id']
# целевая переменная (1)
target_col = ['price']
# итого 1+1+20+3+2+1=28

all_cols =cat_cols+num_cols+time_cols+servis_cols+bin_cols+ target_col
print(f'Кол-во столбцов, для дальнейшей работы после предварительного анализа:= {len(all_cols)}')

In [ ]:
# реализуем выводы из резюме перед детальным анализом по переменным
# берем только левый руль и тольк 26 столбцов
df = df.loc[df['Руль'] == 'Левый', all_cols]
# исключаем Руль из списка признаков для анализа итого остается 25 признаков из которых 2 сервисных (нужно проанализировать 23=25-2)
cat_cols.remove('Руль')
# удаляем дубликаты
df = df.drop_duplicates()
old_len_train = len(df[df['Train']==1])
print('Кол-во строк в трейне:= ', old_len_train)

### Детальный анализ по переменным
---

#### bodyType
начнем с типа кузова bodyType, чтобы убедиться в том что значения по всем маркам и моделям примерно одинаковые или наоборот и тогда убрать нерелевантные типы кузова

In [ ]:
# так как признаков много создаем список проанализированных признаков, чтобы можно было посмотреть, что осталось сделать  
EDA_done_cols=[]

In [ ]:
temp_df_Train = df[df['Train']==1]
temp_df_Test = df[df['Train']==0]
print(f'Всего в тесте типов кузова:= {temp_df_Test.bodyType.nunique()}')
print(f'Всего в трейне типов кузова:= {temp_df_Train.bodyType.nunique()}')

In [ ]:
# посмотрим на них 
print(f'Список 11 типов кузова БМВ из теста:= {temp_df_Test.bodyType.unique()}')
print(f'Список первых 20 из 144 типов кузова из трейна:= {temp_df_Train.bodyType.unique()[:20]}')

In [ ]:
# оставляем только типы кузова как в тесте
list_bodyType_test = list(temp_df_Test.bodyType.unique())
df = df[df['bodyType'].isin(list_bodyType_test)]

Резюме по bodyType:- Категориальный признак, оставили 11 типов кузовов, так как типы кузовов отличающиеся от кузовов в тесте очень специфичны и очень сильно влияют на класс автомобиля по которому по общим закономерностям формируется цена, оставим только типы кузова из теста, а потом после EDA проверим достаточно ли большая выборка трейна остается для моделирования или необходимо парсить другие сайты. Пропусков нет.

In [ ]:
def result_EDA_feature(d_feature, d_train, d_test, d_n_cols, d_EDA_done_cols, d_old_len_train):
    # записываем признак в список проанализированных признаков
    d_EDA_done_cols.append(d_feature)
    d_len_ = len(d_EDA_done_cols)
    print(f'В результате после EDA признака:= {d_feature}, обработано признаков:= {d_len_}, осталось:= {d_n_cols-d_len_}')
    # смотрим как ведет себя трейн
    temp = len(d_train)
    print('Кол-во строк в трейне:= ', temp, '. Убрали на данном шаге:= ', d_old_len_train-temp)
    d_old_len_train = temp
    # проверяем что мы случайно не испортили тест
    print('Кол-во строк в тесте:= ', len(d_test))
    return d_old_len_train, d_EDA_done_cols

In [ ]:
old_len_train, EDA_done_cols = result_EDA_feature('bodyType', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

#### enginePower
забежим вперед и обработаем мощность двигателя потому что она нам потребуется для анализа марок

In [ ]:
# вспомним что по результатам предварительного анализа эта переменная была отнесена к категориальнымu, посмотрим почему
temp_df_Train = df[df['Train']==1]
temp_df_Test = df[df['Train']==0]
display(temp_df_Train.enginePower[:5])
display(temp_df_Test.enginePower[:5])

In [ ]:
# проверим все значения теста по мощности 
temp_df_Test.enginePower.unique()

In [ ]:
# в тесте вытащим значение мощности перед строкой ' N12'
df.loc[df['Train']==0, 'enginePower'] = df[df['Train']==0]['enginePower'].apply(lambda x: int(x.split()[0]))

In [ ]:
# переводим в int64
df['enginePower'] = df['enginePower'].astype('int64')

# перенесем признак в числовые
cat_cols.remove('enginePower')
num_cols.append('enginePower')

In [ ]:
def describe_without_plots(d_name_plot,d_df):
    temp_describe = d_df.describe().copy()
    temp_dict = {}
    temp_dict['кол-во строк'] = len(d_df)
    temp_dict['тип значений'] = d_df.dtype
    temp_dict['кол-во значений'] = temp_describe[0]
    temp_dict['кол-во NaN'] = (d_df.isna()).sum()
    temp_dict['среднее'] = temp_describe[1]
    temp_dict['медиана'] = temp_describe[5]
    temp_dict['мин'] = temp_describe[3]
    temp_dict['макс'] = temp_describe[7]

    temp_df = pd.DataFrame.from_dict(temp_dict, orient='index', columns=[d_name_plot])
    display(temp_df)

    return

In [ ]:
describe_without_plots('enginePower', df[df['Train']==1].enginePower)

In [ ]:
# очень странное минимальное значение можности, давайте посмотрим минимум мощности на тесте 
describe_without_plots('enginePower', df[df['Train']==0].enginePower)